In [1]:
%%html
<marquee style='width:100%; color: blue;'><b> <h1 style="color:Red;font-family:verdana;"> Telecom Churn Prediction </h1></b></marquee>


## Objectives

The main goal of this case study is to develop machine learning models for predicting churn. The predictive models have the following purposes:

* Predicting whether a high-value customer is likely to churn in the near future (i.e., churn phase). This information can assist the company in taking proactive measures such as offering special plans or discounts on recharge to retain customers.

* Identifying significant variables that strongly indicate churn. These variables can provide insights into the reasons why customers choose to switch to other networks.

Based on observations, strategies for managing customer churn are recommended.


### Steps 

1. Data Understanding, Preparation, and Pre-Processing :
   - Data understanding, identification of potentially useful and non-useful attributes and variable importance and impact estimation
   - Data preparation, performing data cleaning, missing values imputation, outlier removal, and column level standardization (for e.g., date, etc.) into one format
   
2. Exploratory Data Analysis :
   - Performing basic preliminary data analysis including finding the correlation between variables and scatter plots to identify relationships between variables
   - Performing advanced data analysis, including plotting relevant heatmaps, histograms, and basic clustering to find patterns in the data
   
3. Feature Engineering and Variable Transformation :
   - Feature engineering and performing one or more methods on attributes that can lead to the creation of a new potentially useful variable; for e.g., day from the date
   - Variable transformation and applying categorical variable transformations to turn into numerical data and numerical variable transformations to scale data
   
4. Model Selection, Model Building, and  Prediction :
   - Identifying the type of problem and making a list of decisive models from all available choices
   - Choosing a training mechanism; for e.g., cross-validation, etc., and tuning hyperparameters of each model
   - Testing each model on the respective model evaluation metric
   - Choosing the best model based on the fit of the data set and output variable
   - Using ensemble options to improve the efficacy based on the evaluation metric stated in the problem

In [2]:
"""loading necessary libraries"""

#import EDA libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#filter warnings and change pd params
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None)


#import data preprocessing 
import sklearn
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer #dealing with missing values -categorial
from sklearn.experimental import enable_iterative_imputer, enable_halving_search_cv
from sklearn.impute import IterativeImputer #dealing with missing values - numeric
import imblearn 
from imblearn.over_sampling import SMOTE #handle class imbalnce
from sklearn.decomposition import PCA #reduce dimensionality
from scipy import stats

#models
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression #provides feature importances
from sklearn.ensemble import RandomForestClassifier #provides feature importances


#metrics and selection
from sklearn.metrics import roc_auc_score, classification_report #roc_auc score and a full classification report
from sklearn.model_selection import train_test_split, StratifiedKFold, HalvingGridSearchCV
from sklearn.pipeline import Pipeline

#time
import datetime

"""random_state for all cases will be 2121"""

seed = 2121

In [3]:
#calculate how long it takes for notebook to run 

notebook_start_time = datetime.datetime.now()

print(f'notebook commenced running at {notebook_start_time}')

notebook commenced running at 2023-05-30 13:08:51.308404


#### Step 1: Data Understanding, Preparation, and Pre-Processing :

- Data understanding, identification of potentially useful and non-useful attributes and variable importance and impact estimation
- Data preparation, performing data cleaning, missing values imputation, outlier removal, and column level standardization (for e.g., date, etc.) into one format

In [4]:
telecom_train = pd.read_csv("train.csv")
telecom_test = pd.read_csv("test.csv")

data_dict = pd.read_csv("data_dictionary.csv")

In [5]:
telecom_train.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,7.527,48.58,124.38,1.29,32.24,96.68,2.33,0.00,0.0,0.0,0.00,0.0,0.00,2.23,0.00,0.28,5.29,16.04,2.33,0.00,0.00,0.00,0.00,0.00,0.00,7.53,16.04,2.61,46.34,124.38,1.01,18.75,80.61,0.0,0.00,0.0,0.0,0.0,0.0,0.0,65.09,204.99,1.01,0.0,0.0,0.0,8.20,0.63,0.00,0.38,0.0,0.0,81.21,221.68,3.63,2.43,3.68,7.79,0.83,21.08,16.91,0.00,0.00,0.00,3.26,24.76,24.71,0.00,7.61,0.21,7.46,19.96,14.96,0.0,0.0,0.0,0.0,0.0,0.0,7.46,27.58,15.18,11.84,53.04,40.56,0.0,0.0,0.66,0.0,0.0,0.0,1.11,0.69,0.00,3,2,2,77,65,10,65,65,10,6/22/2014,7/10/2014,8/24/2014,65,65,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,42.953,0.00,0.00,0.00,0.00,25.99,30.89,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22.01,29.79,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,30.73,31.66,0.00,0.0,0.0,0.00,30.73,31.66,1.68,19.09,10.53,1.41,18.68,11.09,0.35,1.66,3.40,3.44,39.44,25.03,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,3.44,39.44,25.04,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.00,3,4,5,0,145,50,0,145,50,6/12/2014,7/10/2014,8/26/2014,0,0,0,NaN,7/8/2014,NaN,NaN,1.0,NaN,NaN,145.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,Na

In [6]:
#function for summarising dataframe metadata/summary
"""credit: kimtaehun from kaggle - Simple EDA and baseline with LGBM | (AUC >0.94)"""

def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values * 100
    summ['%missing'] = df.isnull().sum().values / len(df)
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').T)
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

In [7]:
telecom_train['churn_probability'].value_counts()

0    62867
1     7132
Name: churn_probability, dtype: int64

strong imbalance in target

In [8]:
data_dict

,Acronyms,Description
0,CIRCLE_ID,Telecom circle area to which the customer belongs to
1,LOC,Local calls within same telecom circle
2,STD,STD calls outside the calling circle
3,IC,Incoming calls
4,OG,Outgoing calls
5,T2T,Operator T to T ie within same operator mobile to mobile
6,T2M,Operator T to other operator mobile
7,T2O,Operator T to other operator fixed line
8,T2F,Operator T to fixed lines of T
9,T2C,Operator T to its own call center


In [9]:
summary(telecom_train)

data shape: (69999, 172)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
id,int64,0,0.000000,69999,0.0,69998.0,0,1,2
circle_id,int64,0,0.000000,1,109.0,109.0,109,109,109
loc_og_t2o_mou,float64,70200,0.010029,1,0.0,0.0,0.0,0.0,0.0
std_og_t2o_mou,float64,70200,0.010029,1,0.0,0.0,0.0,0.0,0.0
loc_ic_t2o_mou,float64,70200,0.010029,1,0.0,0.0,0.0,0.0,0.0
last_date_of_month_6,object,0,0.000000,1,NaN,NaN,6/30/2014,6/30/2014,6/30/2014
last_date_of_month_7,object,39900,0.005700,1,NaN,NaN,7/31/2014,7/31/2014,7/31/2014
last_date_of_month_8,object,73300,0.010472,1,NaN,NaN,8/31/2014,8/31/2014,8/31/2014
arpu_6,float64,0,0.000000,61615,-2258.709,27731.088,31.277,0.0,60.806
arpu_7,float64,0,0.000000,61425,-1289.715,35145.834,87.009,122.787,103.176


Seems there are a  fields with single values additionally, some fields have more than 70% missing data. That is somewhat concerning. May not be enough data to impute. Will look further into IterativeImputer and its limitations